# Market Basket Analysis using Apriori algorithm

I was provided with dataset containing specific transactions of a store. The task is to conduct the Market Basket Analysis and find out what items should be sold together. 

At first, important libraries are loaded, followed by reading the .xlsx file:

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Feature selection

These are the most important features we will focus on:
- InvoiceNo
- Description
- Country 
- Quantity

## 1 Data Check & Cleaning

'One-hot encoding' technique will be used - few fixes throughout the data are needed.
What do we noticed from descriptions below ?
- Nulls in Description
- InvoiceNO Dtype
- Below-zero quantity

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240992 entries, 0 to 240991
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    240992 non-null  object        
 1   StockCode    240992 non-null  object        
 2   Description  240091 non-null  object        
 3   Quantity     240992 non-null  int64         
 4   InvoiceDate  240992 non-null  datetime64[ns]
 5   UnitPrice    240992 non-null  float64       
 6   CustomerID   173555 non-null  float64       
 7   Country      240992 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 14.7+ MB


In [4]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,240992.000000,240992.000000,173555.000000
mean,9.257855,5.122131,15277.911844
std,222.606337,119.749026,1724.425506
min,-74215.000000,0.000000,12346.000000
25%,1.000000,1.250000,13846.000000
50%,3.000000,2.100000,15145.000000
75%,10.000000,4.210000,16814.000000
max,74215.000000,38970.000000,18287.000000


### 1.1 Missing values in Description column

We can see that all the missing values are assigned to United Kingdoms. Such rows containing these NaN values in Description colum are simpy dropped. From further investigation seems that the United Kingdoms transactions represent the majority of all records.


In [5]:
df.isnull().sum()

InvoiceNo          0
StockCode          0
Description      901
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     67437
Country            0
dtype: int64

In [6]:
df[( df['Description'].isnull())]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
237249,557847,84387A,NaN,11,2011-06-23 10:49:00,0.0,NaN,United Kingdom
237286,557851,85130A,NaN,-23,2011-06-23 10:57:00,0.0,NaN,United Kingdom
239229,558032,22443,NaN,-7,2011-06-24 12:14:00,0.0,NaN,United Kingdom
239230,558033,84876B,NaN,-59,2011-06-24 12:15:00,0.0,NaN,United Kingdom


In [7]:
df[( df['Description'].isnull() ) & ( df['Country'] != 'United Kingdom' ) ].shape

(0, 8)

In [8]:
df[ ( df['Country'] == 'United Kingdom' ) ].shape

(221186, 8)

In [9]:
df.dropna(axis=0, subset=['Description'], inplace=True)


- ~~Nulls in Description~~
- InvoiceNO Dtype
- Below-zero quantity

### 1.2 Dtype check

In [10]:
df['InvoiceNo'].dtype

dtype('O')

At first, I tried to convert the column to numeric type. After running the code below, we get error message - there are values in the InvoiceNo column having
non-numeric symbols.

In [11]:
#pd.to_numeric(df['InvoiceNo'])

In [12]:
temp = df['InvoiceNo'].unique().shape[0]
print(f'I checked that now we have {temp} unique invoices.')

I checked that now we have 11610 unique invoices.


In [13]:
without_c_starting = df[~df['InvoiceNo'].astype('str').str.startswith('C')]
pd.to_numeric(without_c_starting['InvoiceNo'])

0         536365
1         536365
2         536365
3         536365
4         536365
           ...  
240987    558120
240988    558120
240989    558120
240990    558120
240991    558120
Name: InvoiceNo, Length: 235700, dtype: int64

I decided to leave the 'InvoiceNo' column as it is. Except values starting with "C", all of the rest represents numeric strings. Even if it should have been treated differently, there is just around 2000 unique transactions (roughly 16% of total unique translation numbers) so I assume that this would not have significant effect on results.

In [14]:
temp = df['InvoiceNo'].unique().shape[0] - without_c_starting['InvoiceNo'].unique().shape[0]
print(f'Unique transaction numbers starting with C: {temp}')

Unique transaction numbers starting with C: 1956



- ~~Nulls in Description~~
- ~InvoiceNo Dtype~
- Below-zero quantity

### 1.3 Below zero Quantity

Some values in Quantity column ale less than zero:

In [15]:
temp = df[df['Quantity'] < 0]['Quantity'].count()
print(f'There is {temp} values < 0 in Quantity column.')

There is 4554 values < 0 in Quantity column.


In [16]:
df[df["Quantity"] < 0] 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
240695,C558112,82482,WOODEN PICTURE FRAME WHITE FINISH,-2,2011-06-26 16:08:00,2.55,17114.0,United Kingdom
240696,C558112,23091,ZINC HERB GARDEN CONTAINER,-1,2011-06-26 16:08:00,6.25,17114.0,United Kingdom
240697,C558112,22796,PHOTO FRAME 3 CLASSIC HANGING,-1,2011-06-26 16:08:00,9.95,17114.0,United Kingdom
240698,C558112,22171,3 HOOK PHOTO SHELF ANTIQUE WHITE,-1,2011-06-26 16:08:00,8.50,17114.0,United Kingdom


I am not sure why those numbers occur. Personally, I would asked management of the store or so, but for the further analysis, they are excluded. Roughly 4500 of total transactions should not make signifficant effect.

In [17]:
df_clean = df[df['Quantity'] > 0]

In [18]:
df.shape[0] - df[df['Quantity'] > 0].shape[0]

4554

- ~~Null values in description~~
- ~~InvoiceNO Dtype~~
- ~~Below-zero quantity~~

### 1.4 Stripping the description column

Further down the analysis, I've noticed that there were some errors when I was filtering by the items purchased.
I treated this fact by stripping the item description, causing removing surrounding whitespaces.

In [19]:
df_clean['Description'] = df_clean['Description'].str.strip()

### Summary 

The dataset is cleaned, prepared for pivoting - procedure that is needed to be executed for using association rules libraries. 

Interesting keypoints: 


In [20]:
total_trans = df_clean.describe().loc['count'][0]
total_invoices = df_clean['InvoiceNo'].unique().shape[0]

print(f'There is {int(total_trans)} transactions executed.')
print(f'There is {total_invoices} unique invoices, each having unique numbers.')

There is 235537 transactions executed.
There is 9491 unique invoices, each having unique numbers.


# Pivoting

This step will be executed along with pivoting the dataset. We need to get it into shape where each unique value in 'Description' column has its own column. Index of such dataframe will be represented as unique InvoiceNo's.

In [21]:
basket = (df_clean.pivot_table(index="InvoiceNo",
                          columns="Description",
                          values="Quantity",
                          fill_value=0) )
basket.head(6)

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,...,amazon sales,did a credit and did not tick ret,found,incorrectly credited C550456 see 47,mailout,on cargo order,rcvd be air temp fix for dotcom sit,returned,to push order througha s stock was,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
536370,0,0,0,0,0.0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
basket['returned'].sum()

4

As we can see by quick look on the resulting dataframe, there are few 'items' that would not make much sense in our analysis. These should not mean a problem, since they will be excluded by apriori algorithm due to low frequencies of occurence.

We can quickly check that our dataframe now contains no below zero values. In the next step, we will convert all values greater than 1 to 1.

In [23]:
temp = {basket[basket < 0].any().sum()}
print(f'Values below zero:{temp}')

Values below zero:{0}


In following step, lambda function is used to replace every value greater than one by one itself. This results in dataset containing zeros and ones only.

In [24]:
basket = basket.apply(lambda x: np.where(x >= 1,1,0))
temp = basket[basket> 1].any().sum()
print(f'Count of Order numbers having any value greater than zero: {temp}. \nThis means now we have just zeros and ones in our table.')

Count of Order numbers having any value greater than zero: 0. 
This means now we have just zeros and ones in our table.


## Apriori Algorithm - Creating frequent Itemsets

Apriori algorithm is used to obtain frequent itemsets. Term itemset describes one item or their combination. These itemsets are represented by each item offered by examined store. 

Such itemsets that are considered frequent must be greater than prior defined threshold. This threshold is specified by parameter min_support of apriori() function, representing the minimal percentage of total records containing particular itemsets. Value of the parameter would be determined by analysis needs, considering proper domain knowledge.

**_Support of an itemset_** = fraction of transactions containing the itemset.

For the whole dataframe, let's pick minimal support threshold of 3%. The result will be a dataframe containing such itemsets having support greater than defined threshold. For executing following algorithms, the mlxtend library was used.

In [25]:
frequent_itemsets = apriori(basket, min_support=0.03, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.046570,(6 RIBBONS RUSTIC CHARM)
1,0.049099,(60 TEATIME FAIRY CAKE CASES)
2,0.036034,(72 SWEETHEART FAIRY CAKE CASES)
3,0.036982,(ALARM CLOCK BAKELIKE GREEN)
4,0.035086,(ALARM CLOCK BAKELIKE PINK)
...,...,...
137,0.036877,"(JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAG..."
138,0.038457,"(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)"
139,0.030450,"(JUMBO STORAGE BAG SUKI, JUMBO SHOPPER VINTAGE..."
140,0.031398,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY..."


In [26]:
frequent_itemsets

,support,itemsets
0,0.046570,(6 RIBBONS RUSTIC CHARM)
1,0.049099,(60 TEATIME FAIRY CAKE CASES)
2,0.036034,(72 SWEETHEART FAIRY CAKE CASES)
3,0.036982,(ALARM CLOCK BAKELIKE GREEN)
4,0.035086,(ALARM CLOCK BAKELIKE PINK)
...,...,...
137,0.036877,"(JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAG..."
138,0.038457,"(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)"
139,0.030450,"(JUMBO STORAGE BAG SUKI, JUMBO SHOPPER VINTAGE..."
140,0.031398,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY..."


## Items often sold together

When looking for items often sold together, the main goal is to find association rules of itemsets. Association rule is basicaly implication - if we know that transaction contain A itemset (**antecedents**), it implies that C (**consequents**) itemset would be also part of the transaction.

We use following metrics to establish Association Rules:

- **Support of a rule** = fraction of transatctions containing A and C together


- **Confidence of a rule** = given A as a part of transaction, what is the probability it cointains C ? Or in other words - fraction of times imtemset C appears in trasactions containing A.


- **Lift of a rule** = measures how much the likelihood of buying C increases knowing that A is also purchased. In other words - how much more likely are these associations than we would expect by chance. If the lift is around 1, that means A and C are rather statistically independent.

We obtain such associations functions of mlxtend library. We can pass an argument of one of metrics above to evaluate the rules. We'll use lift metric for this case, having threshold set to 1.

In [27]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules[pick].sort_values(by='lift', ascending=False)

,antecedents,consequents,support,confidence,lift
2,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.032979,0.552028,13.971466
3,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.032979,0.834667,13.971466
21,(ROSES REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.031398,0.508532,12.870617
20,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.031398,0.794667,12.870617
6,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.045201,0.756614,12.254302
7,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.045201,0.732082,12.254302
8,(HEART OF WICKER SMALL),(HEART OF WICKER LARGE),0.030555,0.455975,9.286818
9,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.030555,0.622318,9.286818
18,(JUMBO STORAGE BAG SUKI),(JUMBO SHOPPER VINTAGE RED PAISLEY),0.030450,0.475329,7.556695
19,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO STORAGE BAG SUKI),0.030450,0.484087,7.556695


The resulting dataframe shows both antecedent and consequent itemsets, along with all three metrics mentioned before. Generally we want all of them to be high. 

We can see that the metrics are quite high. If we have either support and confidence high, it is a good thing. But it could also mean that for example the consequent is being bought often, so it skews the insights. Due to that fact, it is important to check lift metric as well. From the results, it's apparent that there could be some valid rules.


Let's have a look on situation of support threshold set to 2 %.

In [28]:
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']

In [29]:
filter = rules[pick].sort_values(by='confidence')

filter["antecedents"] = filter["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
filter["consequents"] = filter["consequents"].apply(lambda x: list(x)[0]).astype("unicode")

filter = filter[(filter['lift'] > 1.5) & ( filter['confidence'] > 0.2 )].iloc[:50]
filter

,antecedents,consequents,support,confidence,lift
173,REGENCY CAKESTAND 3 TIER,GREEN REGENCY TEACUP AND SAUCER,0.023917,0.202679,4.483968
76,JUMBO BAG RED RETROSPOT,JUMBO BAG TOYS,0.022021,0.208375,6.258500
189,JUMBO BAG RED RETROSPOT,JUMBO STORAGE BAG SUKI,0.022232,0.210369,6.908689
136,REGENCY CAKESTAND 3 TIER,PARTY BUNTING,0.024866,0.210714,2.083218
183,JUMBO BAG RED RETROSPOT,JUMBO BAG PINK POLKADOT,0.022337,0.211366,7.375271
86,JUMBO BAG RED RETROSPOT,RECYCLING BAG RETROSPOT,0.022337,0.211366,4.916847
72,JUMBO BAG RED RETROSPOT,JUMBO BAG SPACEBOY DESIGN,0.022548,0.213360,6.211653
92,JUMBO BAG RED RETROSPOT,WHITE HANGING HEART T-LIGHT HOLDER,0.023391,0.221336,1.665900
137,PARTY BUNTING,REGENCY CAKESTAND 3 TIER,0.024866,0.245833,2.083218
67,JUMBO BAG RED RETROSPOT,JUMBO BAG PINK VINTAGE PAISLEY,0.026551,0.251246,5.444243


In [30]:
filter[(filter['antecedents'].str.contains('PARTY'))
       & ( ~filter['consequents'].str.contains('PARTY'))
      ]

,antecedents,consequents,support,confidence,lift
137,PARTY BUNTING,REGENCY CAKESTAND 3 TIER,0.024866,0.245833,2.083218
139,PARTY BUNTING,WHITE HANGING HEART T-LIGHT HOLDER,0.026130,0.258333,1.944363


Based on the observations, for example if doing any kind of a party, it would be convenieent to sel cake stands and party bunting alltogether.

In [31]:
filter[(filter['antecedents'].str.contains('POPCORN'))
       & ( ~filter['consequents'].str.contains('POPCORN'))
      ]

,antecedents,consequents,support,confidence,lift
91,SMALL POPCORN HOLDER,JUMBO BAG RED RETROSPOT,0.020967,0.340171,3.218906


Combinations of popcorn holder and jumbo bags have also interesting levels of metrics.

## Filtering by specific country

In [32]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta'], dtype=object)

In [33]:
 basket_cz = (df_clean[df_clean['Country'] == 'Czech Republic'].pivot_table(index="InvoiceNo",
                          columns="Description",
                          values="Quantity",
                          fill_value=0) )
basket_cz

Description,3 HOOK HANGER MAGIC GARDEN,BAKING MOULD HEART MILK CHOCOLATE,COLOUR GLASS T-LIGHT HOLDER HANGING,CREAM FELT EASTER EGG BASKET,DECORATION BUTTERFLY MAGIC GARDEN,FELTCRAFT HAIRBAND RED AND BLUE,JIGSAW TREE WITH BIRDHOUSE,MEMO BOARD COTTAGE DESIGN,PINK CREAM FELT CRAFT TRINKET BOX,PINK METAL CHICKEN HEART,ROUND SNACK BOXES SET OF4 WOODLAND,SCOTTIES DESIGN WASHBAG,SET3 BOOK BOX GREEN GINGHAM FLOWER,T-LIGHT HOLDER WHITE LACE,VINTAGE HEADS AND TAILS CARD GAME
InvoiceNo,,,,,,,,,,,,,,,
545072,12,18,48,24,32,24,36,8,24,24,24,24,12,24,12


Unfortunately, for Czech Republic is recorded just a single purchase.

### Germany

After filtering by country, we can observe 'POSTAGE' item showing up repeatedly. We don't have any use for this one, so we drop this column in code below. We can notice that we get more rules using higher min_support threshold - 7% here.

In [34]:
basket_de = (df_clean[df_clean['Country'] == 'Germany'].pivot_table(index="InvoiceNo",
                          columns="Description",
                          values="Quantity",
                          fill_value=0) )
basket_de = basket_de.apply(lambda x: np.where(x >= 1,1,0))

frequent_itemsets_de = apriori(basket_de, min_support=0.07, use_colnames=True)

rules = association_rules(frequent_itemsets_de, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules[pick].head()

,antecedents,consequents,support,confidence,lift
0,(6 RIBBONS RUSTIC CHARM),(POSTAGE),0.085859,0.894737,1.086858
1,(POSTAGE),(6 RIBBONS RUSTIC CHARM),0.085859,0.104294,1.086858
2,(POSTAGE),(COFFEE MUG APPLES DESIGN),0.070707,0.085890,1.214724
3,(COFFEE MUG APPLES DESIGN),(POSTAGE),0.070707,1.000000,1.214724
4,(JAM JAR WITH PINK LID),(POSTAGE),0.090909,0.900000,1.093252


In [35]:
basket_de.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets_de = apriori(basket_de, min_support=0.07, use_colnames=True)
frequent_itemsets_de

rules_de = association_rules(frequent_itemsets_de, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
rules_de[pick].sort_values(by=['lift'], ascending=False).head()

,antecedents,consequents,support,confidence,lift
1,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.090909,0.562500,3.840517
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.090909,0.620690,3.840517
16,"(PLASTERS IN TIN CIRCUS PARADE, ROUND SNACK BO...",(ROUND SNACK BOXES SET OF 4 FRUITS),0.070707,0.700000,3.300000
21,(ROUND SNACK BOXES SET OF 4 FRUITS),"(PLASTERS IN TIN CIRCUS PARADE, ROUND SNACK BO...",0.070707,0.333333,3.300000
11,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.186869,0.880952,3.171429


In [36]:
filter_de = rules_de[pick].sort_values(by='lift')

filter_de["antecedents"] = filter_de["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_de["consequents"] = filter_de["consequents"].apply(lambda x: list(x)[0]).astype("unicode")

filter_de = filter_de[(filter_de['lift'] > 2) & ( filter_de['confidence'] > 0.3 )]

Scaling thresholds and other filtering can produce various results. As we see, by lowering threshold can be obtained great amount of rules, but then it is much harder to go through the list and find reasonable rules.

We can see that it could be beneficial to sell night lights along with snack boxes, probably for parent-customers. For such clients, another idea would be to sell Alarm clocks together with pencil cases. It seems that we are offering more products aimed on kids, so making action offers based on combining stuff used for children's room arrangements and schools stuff would be interesting idea.

There are frequent combinations for example of party ribbons and cakestands. Another rule that seems reasonable would be selling paper plates together with paper cups. This could be possible pattern to focus on - selling goods related to partying together to maximize the volume sold.

Another idea I like is to offer multiple kinds of mugs together. In case of finding out that some specific mug styles tend to be less popular than others for example, we could set up action price when buying these along with the popular ones and get them into circulation.


#### Support threshold: 5%

In [37]:
# decreasing the support threshold to 5%

frequent_itemsets_de = apriori(basket_de, min_support=0.05, use_colnames=True)
frequent_itemsets_de

rules_de = association_rules(frequent_itemsets_de, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']

filter_de = rules_de[pick].sort_values(by='lift')
filter_de["antecedents"] = filter_de["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_de["consequents"] = filter_de["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_de = filter_de[(filter_de['lift'] > 2) & ( filter_de['confidence'] > 0.4 )]

display(filter_de[filter_de["antecedents"].str.contains('RIBBONS')])
display(filter_de[filter_de["antecedents"].str.contains('TOAD')])

,antecedents,consequents,support,confidence,lift
2,6 RIBBONS RUSTIC CHARM,ROUND SNACK BOXES SET OF4 WOODLAND,0.055556,0.578947,2.084211
0,6 RIBBONS RUSTIC CHARM,REGENCY CAKESTAND 3 TIER,0.050505,0.526316,3.256579


,antecedents,consequents,support,confidence,lift
50,RED TOADSTOOL LED NIGHT LIGHT,ROUND SNACK BOXES SET OF4 WOODLAND,0.070707,0.608696,2.191304


#### Support threshold: 3%

In [38]:
# decreasing the support threshold to 3%

frequent_itemsets_de = apriori(basket_de, min_support=0.02, use_colnames=True)
frequent_itemsets_de

rules_de = association_rules(frequent_itemsets_de, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']

filter_de = rules_de[pick].sort_values(by='lift')
filter_de["antecedents"] = filter_de["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_de["consequents"] = filter_de["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_de = filter_de[(filter_de['lift'] > 2) & ( filter_de['confidence'] > 0.4 )]

display(filter_de[filter_de["consequents"].str.contains('PENCIL') & ( filter_de["antecedents"].str.contains('ALARM') )])
display(filter_de[filter_de["consequents"].str.contains('MUG') & ( filter_de["antecedents"].str.contains('MUG') )].head())

,antecedents,consequents,support,confidence,lift
127,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.025253,0.5,11.00
2158,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.020202,0.8,17.60
2170,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.020202,1.0,22.00
2164,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.020202,1.0,22.00
6789,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.020202,1.0,22.00
6793,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.020202,0.8,31.68
6794,ALARM CLOCK BAKELIKE PINK,PENCIL CASE LIFE IS BEAUTIFUL,0.020202,1.0,39.60


,antecedents,consequents,support,confidence,lift
462,CHILDREN'S CIRCUS PARADE MUG,COFFEE MUG APPLES DESIGN,0.025253,0.500000,7.071429
476,CHILDREN'S SPACEBOY MUG,COFFEE MUG APPLES DESIGN,0.020202,0.500000,7.071429
3255,CHILDREN'S CIRCUS PARADE MUG,COFFEE MUG APPLES DESIGN,0.020202,0.571429,8.081633
465,COFFEE MUG PEARS DESIGN,CHILDREN'S CIRCUS PARADE MUG,0.025253,0.555556,11.000000
464,CHILDREN'S CIRCUS PARADE MUG,COFFEE MUG PEARS DESIGN,0.025253,0.500000,11.000000


### United Kingdom

In [39]:
frequent_itemsets_uk = apriori(basket_de, min_support=0.05, use_colnames=True)

rules_uk = association_rules(frequent_itemsets_uk, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']

filter_uk = rules_uk[pick].sort_values(by='lift')
filter_uk["antecedents"] = filter_uk["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_uk["consequents"] = filter_uk["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_uk = filter_uk[(filter_uk['lift'] > 2) & ( filter_uk['confidence'] > 0.4 )]



In [40]:
filter_uk.loc[68].to_frame().T

,antecedents,consequents,support,confidence,lift
68,SET/6 RED SPOTTY PAPER PLATES,SET/6 RED SPOTTY PAPER CUPS,0.0656566,0.866667,10.725


In [41]:
frequent_itemsets_uk = apriori(basket_de, min_support=0.02, use_colnames=True)

rules_uk = association_rules(frequent_itemsets_uk, metric="lift", min_threshold=1)
pick = ['antecedents', 'consequents', 'support', 'confidence', 'lift']

filter_uk = rules_uk[pick].sort_values(by='lift')
filter_uk["antecedents"] = filter_uk["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_uk["consequents"] = filter_uk["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
filter_uk = filter_uk[(filter_uk['lift'] > 2) & ( filter_uk['confidence'] > 0.3 )]
filter_uk
display(filter_uk[filter_uk["consequents"].str.contains('ALARM') & ( filter_uk["antecedents"].str.contains('PENCIL') )].loc[126].to_frame().T)
display(filter_uk[filter_uk["antecedents"].str.contains('RIBBONS') & ( filter_uk["consequents"].str.contains('CAKESTAND') )].loc[94].to_frame().T)

,antecedents,consequents,support,confidence,lift
126,PENCIL CASE LIFE IS BEAUTIFUL,ALARM CLOCK BAKELIKE PINK,0.0252525,0.555556,11


,antecedents,consequents,support,confidence,lift
94,6 RIBBONS RUSTIC CHARM,REGENCY CAKESTAND 3 TIER,0.0505051,0.526316,3.25658


# Conclusion

The dataset analyzed in this notebook consist of transactions of store retail. Purpose of the analysis was to conduct market basket analysis to find out what items are (and should be) commonly sold together. The majority of transactions were closed in UK, thus we can assume that UK is the main region of focus. The task was accomplished by using the Apriori algorithm to identify frequent itemsets by exploring Association rules.

The initial part consists of loading the file and follows up by cleaning and preparation of the data for further handling.
    For market basket analysis procedure itself was convenient to use MLXtend library. It provides an easy to use framework for implementing Apriori algorithm and exploring Association rules, along with other useful functions and concepts used for machine learning and statistical computing. 
    
In the first step of the market basket analysis procedure, the frequent itemsets are found, based on minimal support threshold. Frequent itemsests are established by Algorithm, then they are used to explore Association rules. We can get various results by tuning the parameters like minimal threshold mentioned before and by shaping the rules by specific metrics. At this part, it is convenient to have a good domain knowledge to know what to look for.



#### Results 

At first I created itemset pairs based on data from the whole dataset. The initial minimal support threshold 3% gives already relatively short list of possible combinations. We can get more results by further lowering the threshold. 
By filtering the data for higher values of lift metrics I assume that interesting combinations to be sold together could be for example:

- **Party bunting & Cakestands**

- **Party bunting & Hanging hearth-shaped light holder**

- **Popcorn holder & Retrospot Jumbo bag**

Each transaction has the record of the country where it was closed. Customers from different countries can have different demand, so when we inspect specific country transactions, there could be different itemsets frequently occuring. Further exploration of this may be interesting idea to tailor up the offers of goods for each country specifically. But throughout the analysis I noticed that the vast majority of transactions were closed as UK transactions, so results from countries having relatively small number of transactions can be misleading.  By filtering countries to Germany and UK, I picked few another rules:

- **Red Toadstool led night ligh & Round Woodland snack boxes**

- **Rustic ribbons & Cakestand**

- **Alarm clocks & Pencil cases**

- **Plastic Cups & Plastic Plates**


Such combinations seems reasonable. **People preparing to party of any kind would buy stuff goods like paper cups and plates, or cakestands and maybe some light holders and candles for decoration at once**. This would be clever way to set up some good action offers. 

Another possible combinations would contain **things related to school and kids**, as alarm clocks, and pencil cases, mentioned above, along with lunch bags and boxes.

From further explorations, we can see that different mugs are sold together repeatedly. Convenient idea how to boost total sales could be finding mug styles that are the most popular, and create an offer like 2 + 1 free by selling the less popular ones with it for dicount, to get these less demanded into circulation.

In the end, I am not sure how to interpret the rules containing antecedents and consequents being almost the same items. Maybe these items were already offered as a part of some discount when bought together, thus the frequent appearance in the dataset. Making assumptions would require consultation with a person from the field. Another possible explanation that comes on my mind is that our customers are store owners who tend to buy in bulk picking similar type of items at once to keep the variety of their sortiment.